# Chapter 13 : Using Inheritance and Dynamic Binding


## 13.1 Inheritance


So we're modling students now. Renaming Student_info to core, and reserving Student_info as a `name` that denotes any kind of student.

In [1]:
#include <iostream>
#include <algorithm>
#include <string>
#include <vector>

In [2]:
.rawInput

Using raw input


In [3]:
double grade(double midterm, double final, const std::vector<double> C){
    typedef std::vector<int>::size_type s_type;
    double hw_grade = std::accumulate(C.begin(), C.end(), 0) / C.size();
    return (.3*midterm + .3*final + .4*hw_grade);
}

std::istream& read_hw(std::istream& in, std::vector<double>& hw) {
  if (in) {
    // get rid of previous contents
    hw.clear();

    // read homework grades
    double x;
    while (in >> x) hw.push_back(x);

    // clear the stream so that input will work for the next student
    in.clear();
  }

  return in;
}

class Core {
public:
    Core();
    Core(std::istream&);
    std::string name() const;
    virtual double grade() const;
    virtual std::istream& read(std::istream&);
protected:
    std::istream& read_common(std::istream&);
    double midterm, final;
    std::vector<double> homework;
private:
    std::string n;
};

std::string Core::name() const { return n; }
double Core::grade() const
{
    //use function grade in global namespace
    return ::grade(midterm, final, homework);
}
std::istream& Core::read_common(std::istream& in){
    in >> n >> midterm >> final;
    return in;
}
std::istream& Core::read(std::istream& in){
    read_common(in);
    read_hw(in, homework);
    return in;
}

In [4]:
class Grad : public Core {
public:
    Grad();
    Grad(std::istream&);
    double grade() const;
    std::istream& read(std::istream&);
private:
    double thesis;
};

std::istream& Grad::read(std::istream& in)
{
    read_common(in);
    in >> thesis;
    read_hw(in, Core::homework);  //being explicit that this is in the base class(I think this is good fight me)
    return in;
}


//remember std::min is in <algorithm>
double Grad::grade() const{
    return std::min(Core::grade(), thesis);
}  // recall, :: is scope operator. Putting together the dots lol
// using the scope operator is __essential__ here,
// because we would have been recursively calling ourselves with the default behavior


In [5]:
.rawInput

Not using raw input


So we all know what protected is.
```
The use of public in public Core says that the fact that Grad inherits from Core is part of its
interface, rather than part of its implementation.
```

### 13.1.3 Inheritance and Constructors

Derived objects are constructed by:
1. Allocating space for the entire object (base-class members as well as derived members)
2. Calling the base-class constructor to initialize the base-class parts of the object
3. Initializing the members of the derived class as directed by the constructor initializer
4. Executing the body of the derived-class constructor, if any

I'm guessing the destructor works in reverse

## 13.2 Polymorphism and virtual functions

We still need to add a compare feature to Core to recreate Student_info's functionality.


In [6]:
bool compare(const Core& c1, const Core& c2){
    return c1.name() < c2.name();
}

//this can be used to compare both Core and Grad records b/c u kno inheritance



```
Because every Grad object has a Core part, we can bind compare 's reference parameters to the
Core portions of Grad objects, exactly as we can bind them to plain Core objects. Similarly, we
could have defined compare to operate on pointers to Core or on objects of type Core (as opposed
to a reference to Core ). In either case, we could still call the function on behalf of a Grad object. If
the function took pointers, we could pass a pointer to Grad . The compiler would convert the Grad*
to a Core* , and would bind the pointer to the Core part of the Grad object. If the function took a
Core object, then what would be passed is just the Core portion of the object. There can be striking
differences in behavior, depending on whether we pass an object itself, or a reference or pointer to
the object—as we shall now see.
```
\-> compiler info here is most useful

If we wanted to implement a version of compare as such:
```
bool compare_grades(const Core& c1, const Core& c2)
{
return c1.grade() < c2.grade();
}
```

we would be using Core's method, not the new grading scheme, because we haven't defined virtual yet(clearly!). Note that thie defers which method we want to be decided at run-time(i.e. virtual adds a run-time component).


```
the fact that a function is virtual is inherited, so we need
not repeat the virtual designation on the declaration of grade within the Grad class
```
I didn't know this.
* If implementation of method is separate from declaration, implementation doesn't need `virtual`


For the purposes of inheritence, the type of an object is fixed: it's known at compile-time. However, a refrence or poniter is what makes the virtual mechanism work. (fuck what does object mean again?)

i.e. this version of compare_grades wouldn't work
```
bool compare_grades(const Core& c1, const Core& c2)
{
return c1.grade() < c2.grade();
}
```

If we passed a Grad object in this implementation, we would only be passing the *base* part of the object
\-> The Grad object will be passed to its Core part, and a copy of that portion will be passed to the function
\-> Thus the calls are statically bound, i.e. at compile time.

so this is where the theme of `dynamic binding` vs `static binding` enters again

So `Core &r` is its own special type like `Core *r` ? i.e. I thought `Core &r` was the same as `Core r` but you're just using that `&r` notation when you're swapping addresses, like `Core &r = &b`, or `Core r; &r=&b;`


## 13.3 Using Inheritance to Solve Our Problem
We want to solve that grading problem, but now using inheritence



### Containers of a virtually unknown type
* Use pointer type for  the sweet inheritence
  * this however delegates elsewhere to use `new` and manage memory
* Rewrite compare function for sort to use ptrs

* also remember to use virtual destructors pls
  * the fact that a destructor is virtual is also inherited (but often it is harmlessly included in derived types anyway for clearer code)


## 13.4 A simple handle class

Problems with approach so far:
* Users have to remember to allocate/free space.
* Code is constantly dereferencing pointers

We'll make a *handle class*, that can use both kind of objects


# TO CONTINUE PAGE 342/343

In [ ]:
class Student_info {
public:
  //constructors and copy control
  Student_info():cp(0) { }
  Student_info(std::istream& is): cp(0) { read(is); }
  Student_info(const Student_info&);
  Student_info& operator=(const Student_info&);
  ~Student_info(){ delete cp; }
  
  //operations
  std::istream& read(std::istream&);
  
  std::string name() const {
    if (cp) return cp->name();
    else throw std::runtime_error("uninitialized Student");
  }
  double grade() const {
    if (cp) return cp->grade();
    else throw std::runtime_error("uninitialized Student");
  }
  static bool compare(const Student_info &s1, const Student_info& s2) {
    return s1.name() < s2.name();
  }
private:
  Core* cp;
}

std::istream& Student_info::read(std::istream& is) {
  delete cp;
  
  char ch;
  is >> ch;
  
  if (ch == 'U') {
    cp = new Core(is);
  } else {
    cp = new Grad(is);
  }
  
  return is;
}

//lol this kind of class is the basis of all memory management techniques
//does this class add any overhead at all though vs. just the Core* cp object?(clearly all functions only need to exist in memory once),
//but is there any extra memory allocated per-instance of Core ?
//there is one extra call per method, but that doesn't really matter

